In [1]:
import numpy as np
from makiflow.models.ssd.ssd_utils import jaccard_index
from makiflow.metrics.od_utils import nms

In [ ]:

import tqdm

In [ ]:
model = Builder.ssd_from_json('/home/ikilbas/work_dir/research/gradient_engineering/model.json')

In [ ]:
#import os
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
from makiflow.tools import XmlParser

In [3]:
parser = XmlParser()

In [4]:
d = parser.parse_all_in_dict(source_path='/mnt/data/voc_test/VOCdevkit/VOC2012/Annotations')

100%|██████████| 5138/5138 [00:01<00:00, 2970.36it/s]


In [5]:
import numpy as np
import json

In [9]:
count = dict([(key,0) for key,_ in name2class.items()])

In [10]:
import copy 

In [11]:
count_train = copy.deepcopy(count)

In [12]:
for i in range(len(d)):
    info = d[i]
    obj = info['objects']
    for dic in obj:
        count_train[dic['name']] += 1

In [13]:
count_train

{'aeroplane': 0,
 'bottle': 0,
 'motorbike': 0,
 'train': 0,
 'sheep': 0,
 'person': 7326,
 'bicycle': 0,
 'boat': 0,
 'pottedplant': 0,
 'diningtable': 2,
 'bird': 0,
 'horse': 0,
 'chair': 2,
 'car': 0,
 'sofa': 0,
 'tvmonitor': 0,
 'cow': 0,
 'bus': 0,
 'cat': 0,
 'dog': 0}

In [ ]:
count_test = copy.deepcopy(count)
count_train = copy.deepcopy(count)

In [ ]:
for i in range(len(test_set)):
    info = test_set[i]
    obj = info['objects']
    for dic in obj:
        count_test[dic['name']] += 1

In [ ]:
for i in range(len(train_set)):
    info = train_set[i]
    obj = info['objects']
    for dic in obj:
        count_train[dic['name']] += 1

In [ ]:
with open('data/testset_2007.json', 'w') as f:
    json.dump(d, f)

In [8]:
with open('data/classes.json', 'r') as f:
    name2class = json.load(f)

In [ ]:
d[0]

In [ ]:
class_name_to_num = {}
i = 1
for cl in parser.classes:
    class_name_to_num[cl] = i
    i += 1

In [ ]:
name2class = {'train': 1,
 'tvmonitor': 2,
 'aeroplane': 3,
 'pottedplant': 4,
 'diningtable': 5,
 'cow': 6,
 'sheep': 7,
 'person': 8,
 'horse': 9,
 'motorbike': 10,
 'bird': 11,
 'car': 12,
 'sofa': 13,
 'boat': 14,
 'cat': 15,
 'chair': 16,
 'bicycle': 17,
 'bottle': 18,
 'bus': 19,
 'dog': 20}

In [ ]:
preparator = DataPreparator(d, class_name_to_num, '/mnt/data/voc_test/VOCdevkit/VOC2012/JPEGImages/')

In [ ]:
preparator.load_images()

In [ ]:
preparator.resize_images_and_bboxes((300, 300))

In [ ]:
imgs = preparator.normalize_images()

In [ ]:
import makiflow as mf
mf.set_main_gpu(2)

In [ ]:
import tensorflow as tf
model.set_session(tf.Session())


In [ ]:
model.load_weights('/home/ikilbas/work_dir/research/gradient_engineering/object_detection/ssd_pipeline/reg_test/focal.ckpt')

In [ ]:
import matplotlib.pyplot as plt 

In [ ]:
d[128]

In [ ]:
plt.imshow(imgs[128])

In [ ]:
preds = []
for i in range(len(imgs)):
    # preds = [confidences, locs]
    # `confidences` shape is [batch_sz, total_predictions, num_classes]
    # `locs` shape is [batch_sz, total_predictions, 4]
    preds += [model.predict([imgs[i]])]

In [ ]:
from makiflow.metrics.od_metrics import mAP_maki_supported

In [ ]:
mAP_maki_supported(preds, iou_threshold=0.5, conf_threshold=0.5, test_dict=d, name2class=name2class)

In [ ]:
filtered_preds = []
for pred in preds:
    # bboxes, classes, confidences
    # `bboxes` is a list of ndarrays
    # `classes` is a list of ints
    # `confidences` is a list of floats
    filtered_preds += [nms(pred[1][0], pred[0][0], conf_threshold=0.5, iou_threshold=0.5)]

In [ ]:
filtered_preds[0]

In [ ]:
([np.array([0, 0, 0, 0], dtype=np.float32)], [-1], [0.0])

In [ ]:
def clear_filtered_preds(filtered_preds):
    # Deletes empty predictions
    сleared = []
    for filtered_pred in filtered_preds:
        if len(filtered_pred[0]) != 0:
            сleared += [filtered_pred]
        else:
            сleared += [([np.array([1, 1, 1, 1], dtype=np.float32)], [-1], [0.0])]
    return сleared

In [ ]:
filtered_preds = clear_filtered_preds(filtered_preds)

In [ ]:
def merge_nms_result(filtered_pred):
    boxes = np.vstack(filtered_pred[0])
    classes = np.array(filtered_pred[1])
    confs = np.array(filtered_pred[2])
    return boxes, classes, confs

In [ ]:
pred_boxes = []
pred_cs = []
pred_ps = []
for filtered_pred in filtered_preds:
    boxes, classes, confs = merge_nms_result(filtered_pred)
    pred_boxes += [boxes]
    pred_cs += [classes]
    pred_ps += [confs]

In [ ]:
def parse_dicts(data_dicts, name2class):
    # `data_dict` is a list of dictionaries.
    # The dictionaries contain several fields:
    # 'filename' - name of the file;
    # 'folder' - name of the folder;
    # 'size' - size of the pictire: (channels, width, height);
    # 'objects' - list of dictionaries that contain info about the 
    # objects on the image:
    # [{'name' : 'object_name', 'box': [x1, x2, x3, x4]}]
    true_boxes = []
    true_classes = []
    for image_dict in data_dicts:
        # Collect the bboxes and stack them together
        image_boxes = []
        image_classes = []
        for object_dict in image_dict['objects']:
            image_boxes += [np.array(object_dict['box'], dtype=np.float32)]
            image_classes += [name2class[object_dict['name']]]
        true_boxes += [np.vstack(image_boxes)]
        true_classes += [np.array(image_classes, dtype=np.int32)]
    return true_boxes, true_classes

In [ ]:
true_boxes, true_classes = parse_dicts(d, name2class=name2class)

In [ ]:
true_classes

In [ ]:
def compute_ap(recall, precision):
    """ Compute the average precision, given the recall and precision curves.
    Code originally from https://github.com/rbgirshick/py-faster-rcnn.
    # Arguments
        recall:    The recall curve (list).
        precision: The precision curve (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.0], recall, [1.0]))
    mpre = np.concatenate(([0.0], precision, [0.0]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap


In [ ]:
def compute_tps(pred_boxes, pred_classes, true_boxes, true_classes, iou_th=0.5):
    """
    Computes True Positives for the given predictions.
    
    Parameters
    ----------
    pred_boxes : ndarray
        Array of the predicted bounding boxes.
    pred_classes : ndarray
        Array of the predicted classes for `pred_boxes`.
    true_boxes : ndarray
        Array of the true bouding boxes.
    true_classes : ndarray
        Array of the true classes for `true_boxes`.
        
    Returns
    -------
    ndarray
        Array of shape [len(pred_boxes)] where ith element equals 1 if ith bbox is correct
        and 0 otherwise.
        
    """
    tps = np.zeros(len(pred_boxes))
    for i, true_box in enumerate(true_boxes):
        ious = jaccard_index(pred_boxes, np.array([true_box] * len(pred_boxes)))
        iou_match = ious > iou_th
        class_match = pred_classes == true_classes[i]
        tp_indices = iou_match * class_match
        if np.max(tp_indices) != False:
            tps[np.argmax(tp_indices)] = 1
    return tps

In [ ]:
def ap_per_class(tp, conf, pred_cls, target_cls):
    """ Compute the average precision, given the recall and precision curves.
    Source: https://github.com/rafaelpadilla/Object-Detection-Metrics.
    # Arguments
        tp:    True positives (list).
        conf:  Objectness value from 0-1 (list).
        pred_cls: Predicted object classes (list).
        target_cls: True object classes (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """

    # Sort by objectness
    i = np.argsort(-conf)
    tp, conf, pred_cls = tp[i], conf[i], pred_cls[i]

    # Find unique classes
    unique_classes = np.unique(target_cls)

    # Create Precision-Recall curve and compute AP for each class
    ap, p, r = [], [], []
    for c in tqdm.tqdm(unique_classes, desc="Computing AP"):
        i = pred_cls == c
        n_gt = (target_cls == c).sum()  # Number of ground truth objects
        n_p = i.sum()  # Number of predicted objects

        if n_p == 0 and n_gt == 0:
            continue
        elif n_p == 0 or n_gt == 0:
            ap.append(0)
            r.append(0)
            p.append(0)
        else:
            # Accumulate FPs and TPs
            fpc = (1 - tp[i]).cumsum()
            tpc = (tp[i]).cumsum()

            # Recall
            recall_curve = tpc / (n_gt + 1e-16)
            r.append(recall_curve[-1])

            # Precision
            precision_curve = tpc / (tpc + fpc)
            p.append(precision_curve[-1])

            # AP from recall-precision curve
            ap.append(compute_ap(recall_curve, precision_curve))

    # Compute F1 score (harmonic mean of precision and recall)
    p, r, ap = np.array(p), np.array(r), np.array(ap)
    f1 = 2 * p * r / (p + r + 1e-16)

    return p, r, ap, f1, unique_classes.astype("int32")


In [ ]:
def mAP(pred_boxes, pred_cs, pred_ps, true_boxes, true_cs, iou_th=0.5):
    """
    Parameters
    ----------
    pred_boxes : list
        List of ndarrays of shape [total_preds, 4]. Contains predicted bboxes for one image.
    pred_cs : list
        List of ndarrays of shape [total_preds]. Contains classes for `pred_boxes`.
    pred_ps : list
        List of ndarrays of shape [total_preds]. Contains confidences for `pred_cs`.
    true_boxes : list
        List of ndarrays of shape [total_preds, 4]. Contains true bboxes for one image.
    true_cs : list
        List of ndarrays of shape [total_preds]. Contains classes for `true_boxes`.
    
    """
    
    num_images = len(pred_boxes)
    all_tps = []
    for i in range(num_images):
        all_tps += [compute_tps(
            pred_boxes=pred_boxes[i],
            pred_classes=pred_cs[i],
            true_boxes=true_boxes[i],
            true_classes=true_cs[i],
            iou_th=iou_th
        )]
        
    all_tps = np.concatenate(all_tps, axis=0)
    pred_cs = np.concatenate(pred_cs, axis=0)
    pred_ps = np.concatenate(pred_ps, axis=0)
    true_cs = np.concatenate(true_cs, axis=0)
    
    p, r, ap, f1, unique_classes = ap_per_class(
        tp=all_tps,
        conf=pred_ps,
        pred_cls=pred_cs,
        target_cls=true_cs
    )
    return p, r, ap, f1, unique_classes

In [ ]:
p, r, ap, f1, unique_classes = mAP(pred_boxes, pred_cs, pred_ps, true_boxes, true_classes)

In [ ]:
ap

In [ ]:
f1

In [ ]:
unique_classes

In [ ]:
p

In [ ]:
# [num_predictions]
# preds = [[conf, locs]]
# conf = [batch_sz, num_classes]
len(preds[0])

In [ ]:
a, b = preds[0]

In [ ]:
splitted_preds = []
for pred in preds:
    splitted_preds += [[p, l] for p, l in zip(pred[0], pred[1])]

In [ ]:
splitted_preds[1][0].shape

In [ ]:
for i in range(len(splitted_preds)):
    splitted_preds[i][0], splitted_preds[i][1]

In [ ]:
[nms(preds[0][1][0], preds[0][0][0], conf_threshold=0.5, iou_threshold=0.5)]

In [ ]:
# Split the prediction array 
filtered_preds = []
for pred in tqdm.tqdm(splitted_preds):
    # bboxes, classes, confidences
    # `bboxes` is a list of ndarrays
    # `classes` is a list of ints
    # `confidences` is a list of floats
    filtered_preds += [nms(pred[1], pred[0], conf_threshold=0.5, iou_threshold=0.5)]

In [ ]:
filtered_preds_ = []
for confidences, localisations in preds:
    for image_confs, image_locs in zip(confidences, localisations):
        # bboxes, classes, confidences
        # `bboxes` is a list of ndarrays
        # `classes` is a list of ints
        # `confidences` is a list of floats
        filtered_preds_ += [nms(image_locs , image_confs, conf_threshold=0.5, iou_threshold=0.5)]

In [ ]:
filtered_preds[1]

In [ ]:
filtered_preds[1]

In [ ]:
filtered_preds_[0]

In [ ]:
filtered_preds = []
for confidences, localisations in preds:
    for image_confs, image_locs in zip(confidences, localisations):
        # bboxes, classes, confidences
        # `bboxes` is a list of ndarrays
        # `classes` is a list of ints
        # `confidences` is a list of floats
        filtered_preds += [nms(image_locs , image_confs, conf_threshold=0.5, iou_threshold=0.5)]
# Clear the NMS results from empty predictions
filtered_preds = clear_filtered_preds(filtered_preds)
# Convert NMS results to separate lists of numpy arrays
pred_boxes = []
pred_cs = []
pred_ps = []
for filtered_pred in filtered_preds:
    boxes, classes, confs = merge_nms_result(filtered_pred)
    pred_boxes += [boxes]
    pred_cs += [classes]
    pred_ps += [confs]

# PROCESS THE GROUND TRUE LABELS
true_boxes, true_classes = parse_dicts(d, name2class=name2class)

# COMPUTE THE mAP
p, r, ap, f1, unique_classes = mAP(
    pred_boxes,
    pred_cs,
    pred_ps,
    true_boxes,
    true_classes,
    iou_th=0.5
)

In [ ]:
p, r, ap, f1, unique_classes

In [ ]:
true_classes